# Predicting dispersion coefficient
This example will show how to find the dispersion coefficient of a porous medium.

In [1]:
import numpy as np
import openpnm as op
import matplotlib.pyplot as plt
from scipy import special
from scipy.optimize import curve_fit
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
np.random.seed(10)
np.set_printoptions(precision=5)

## Generating Network
A 2D 40 X 40 ``Cubic`` network is generated with a spacing of $10^{-4}$m, but a 3D network would work as well.

In [5]:
shape = [40, 40, 1]
pn = op.network.Cubic(shape=shape, spacing=1e-4)
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()
water = op.phase.Water(network=pn)
water.add_model_collection(op.models.collections.physics.basic)
water.regenerate_models()

## Defining Effective Pore Volume

The accumulation of mass in the network occurs only in the pores, where the concentration is solved.  In order for mass  accumulate properly, it is necessary to assign the throat volumes to their surrounded throats.  This creates an effective pore volume.  We can define this in a custom pore-scale model, making use of the ``numpy.add.at`` function, to add 1/2 the volume of each throat to its neighboring pores.  

In [3]:
def effective_pore_volume(target, throat_volume='throat.volume', pore_volume='pore.volume'):
    Pvol = pn['pore.volume']
    Tvol = pn['throat.volume']
    Vtot = Pvol.sum() + Tvol.sum()
    np.add.at(Pvol, pn.conns[:, 0], pn['throat.volume']/2)
    np.add.at(Pvol, pn.conns[:, 1], pn['throat.volume']/2)
    assert np.isclose(Pvol.sum(), Vtot)  # Ensure total volume has been added to Pvol
    return Pvol

In [6]:
pn.add_model(propname='pore.effective_volume', model=effective_pore_volume)

## Perform Stokes flow
The advection diffusion algorithm assumes a velocity field. Therefore, Stokes flow in the pore netwok is solved. The ``StokesFlow`` algorthm is solved prior to running the ``AdvectionDiffusion`` algorthim. For more information there is a seperate tutorial on Stokes Flow.

In [7]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('back'), values=50.0)
sf.set_value_BC(pores=pn.pores('front'), values=0)
sf.run();

The results obtained from the StokesFlow algorthim must be attached to the water phase.

In [13]:
water.update({'pore.pressure':sf['pore.pressure']})

## Add Diffusive Conductance Model

In [15]:
mod = op.models.physics.ad_dif_conductance.ad_dif
water.add_model(propname='throat.ad_dif_conductance', model=mod, s_scheme='powerlaw')

## Define Transient Advection Diffusion
An algorthim for transient advection diffusion is defined here. It is assigned to the network and the phase, and will be able to retrieve all information that will be needed.

In [16]:
ad = op.algorithms.TransientAdvectionDiffusion(network=pn, phase=water) 

The Dirichlet boundary conditions and the inital conditions are next defined as follows. If the inital condition is not defined then it is assumed to be zero, so it is redundunt in this case. The boundary conditions can be defined as ``value``, ``outflow``, and ``rate``.

In [17]:
inlet  = pn.pores('back') 
outlet = pn.pores('front')
ad.set_value_BC(pores=inlet, values=1.0)
ad.set_outflow_BC(pores=outlet)

## Setup the Transient Algorithim
The settings of the transient algorthim can be updated here. We first define the time span:

In [18]:
tspan = (0, 100)
saveat = 5

We must also tell the algorithm to use the effective pore volume rather than the default which is just 'pore.volume'

In [19]:
ad.settings['pore_volume'] = 'pore.effective_volume'

In [32]:
print(ad)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.algorithms.TransientAdvectionDiffusion : trans_ad_01
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.bc.outflow                                  40 / 1600 
2     pore.bc.rate                                      0 / 1600 
3     pore.bc.value                                    40 / 1600 
4     pore.concentration                             1600 / 1600 
5     pore.ic                                        1600 / 1600 
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Labels                                        Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.all                                      1600      
2     throat.al

The algorthim than can be run, but we must pass the initial conditions (could be a scalar or an array), time span, and optionally the intervals at which the solution is desired to be stored.

In [31]:
soln = ad.run(x0=0, tspan=tspan, saveat=saveat)

## Solving for the Predicti

The following equation given by Fried (1971) is used to solve the longitudinal dispersion coefficient:

$$\frac{C}{C_{0}} = \frac{1}{2}erfc\Bigl(\frac{x-Ut}{2(D_{L}t)^{\frac{1}{2}}}\Bigr)+\frac{1}{2}exp\Bigl(\frac{Ux}{D_{L}}\Bigl)erfc\Bigr(\frac{x+Ut}{2(D_{L}t)^{\frac{1}{2}}}\Bigr)$$

Where $x$ is the length between the inlet and the outlet, $t$ is the time, $D_{L}$ is the longitudinal dispersion coefficient, $U$ is the average pore velocity, $C_{0}$ is the inlet concentration, and $C$ is the concentration at the given time. Since we defined the inlet concentration as being equal to 1, solving for $C$ is effictivly equal to solving for $\frac{C}{C_{0}}$. ``erfc`` is the complementary error function, which is imported from ``scipy``.



In [21]:
q_throat = sf.rate(throats=pn.Ts, mode='single')
A_throat = pn['throat.cross_sectional_area']
v_throat = q_throat/A_throat
v_pred = sum(q_throat*v_throat)/sum(q_throat)

def elution(step,v,DL):
    x = 40*1e-4
    el1 = 0.5*(special.erfc((x-step*v)/(2*(DL*step)**(1/2))))
    el2 = 0.5*np.exp(v*x/DL)
    el3 = special.erfc((x+step*v)/(2*(DL*step)**(1/2)))
    return el1+el2*el3

In [22]:
g = [v_pred, 1e-3]
xdata = [float(x) for x in soln.t]
ydata = c_avg

popt, pcov = curve_fit(elution, xdata, ydata, p0=g)
disp_coeff = popt[1]
v_fit = popt[0]

print('Dispersion Coefficient = ', "{0:.4E}".format(disp_coeff), ' m^2/s')
print('v_pred = ', "{0:.4E}".format(v_pred), ' m/s')
print('v_fitted = ', "{0:.4E}".format(v_fit), ' m/s')

AttributeError: 'SolutionContainer' object has no attribute 't'

In [ ]:
el = np.zeros(len(ydata))
for i in range(len(ydata)):
    el[i] = elution(xdata[i], popt[0], popt[1])

fig, ax = plt.subplots()
ax.plot(xdata, ydata, label="simulation")
ax.plot(xdata, el, ".", label="fitted")
ax.legend()
ax.set_xlabel('time (s)')
ax.set_ylabel('concentration');